## Import libbrary

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import math
import os

In [2]:
relative_path = "/content/sample_data/Data"
CSV_files = os.listdir(relative_path)

## Meger File CSV

In [3]:
dataFrame = []
for file in CSV_files:
  read = pd.read_csv(os.path.join(relative_path, file))
  dataFrame.append(read)
df = pd.concat(dataFrame)

In [4]:
df_copy = df.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186850 entries, 0 to 17660
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 10.0+ MB


In [6]:
df["Quantity Ordered"] = pd.to_numeric(df["Quantity Ordered"],  errors='coerce')
df = df.dropna(axis='rows')
df["Price Each"] = pd.to_numeric(df["Price Each"] , errors = 'raise')
df["Order Date"] = pd.to_datetime(df["Order Date"] , errors = 'raise')


<ipython-input-6-b625410b8435>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Order Date"] = pd.to_datetime(df["Order Date"] , errors = 'raise')


In [7]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,248151,AA Batteries (4-pack),4.0,3.84,2019-09-17 14:44:00,"380 North St, Los Angeles, CA 90001"
1,248152,USB-C Charging Cable,2.0,11.95,2019-09-29 10:19:00,"511 8th St, Austin, TX 73301"
2,248153,USB-C Charging Cable,1.0,11.95,2019-09-16 17:48:00,"151 Johnson St, Los Angeles, CA 90001"
3,248154,27in FHD Monitor,1.0,149.99,2019-09-27 07:52:00,"355 Hickory St, Seattle, WA 98101"
4,248155,USB-C Charging Cable,1.0,11.95,2019-09-01 19:03:00,"125 5th St, Atlanta, GA 30301"


In [8]:
df.describe()

,Quantity Ordered,Price Each,Order Date
count,185950.000000,185950.000000,185950
mean,1.124383,184.399735,2019-07-18 21:54:38.887550464
min,1.000000,2.990000,2019-01-01 03:07:00
25%,1.000000,11.950000,2019-04-16 21:05:15
50%,1.000000,14.950000,2019-07-17 20:40:30
75%,1.000000,150.000000,2019-10-26 08:14:00
max,9.000000,1700.000000,2020-01-01 05:13:00
std,0.442793,332.731330,NaN


In [9]:
df.to_csv("Data_Year.csv",index= False)

## Handle Data and answers question

### Question 1

In [10]:
df_months = df_copy[["Price Each","Order Date"]]
df_months["Order Date"] = df_months['Order Date'].str[0:2]

<ipython-input-10-6e0f81ac926b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_months["Order Date"] = df_months['Order Date'].str[0:2]


In [11]:
df_months['Price Each'] = pd.to_numeric(df_months['Price Each'], errors='coerce')

<ipython-input-11-752643f02675>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_months['Price Each'] = pd.to_numeric(df_months['Price Each'], errors='coerce')


In [12]:
sum_month = df_months.groupby("Order Date").sum()

In [13]:
sum_month = sum_month.drop(index='Or')

In [14]:
fig1 = px.bar(sum_month)
fig1.update_layout(
    title="Sales by month",
    xaxis_title="Month",
    yaxis_title="Sales in USD ($)",
    legend_title="Value USD ($)",
)
fig1.show()

In [15]:
value_earning = sum_month.max()
month_max_value = sum_month.idxmax()

In [16]:
print(value_earning)

Price Each    4588415.41
dtype: float64


#### The best month for sales

#### Earned that month

In [17]:
print(month_max_value)

Price Each    12
dtype: object


### Question 2

In [18]:
df_city = df[["Quantity Ordered","Purchase Address"]]

In [19]:
df_city['City'] = df_city['Purchase Address'].str.split(',', expand=True)[1]

<ipython-input-19-18b4145585d3>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
df_city.drop(columns='Purchase Address',inplace=True)

<ipython-input-20-8724954d8076>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
df_city = df_city.groupby("City").sum()

In [22]:
df_city

,Quantity Ordered
City,
Atlanta,16602.0
Austin,11153.0
Boston,22528.0
Dallas,16730.0
Los Angeles,33289.0
New York City,27932.0
Portland,14053.0
San Francisco,50239.0
Seattle,16553.0


In [23]:

fig2 = px.bar(df_city)
fig2.update_layout(
    title="Sales by Cites",
    xaxis_title="City",
    yaxis_title="Sales in USD ($)",
    legend_title="Value USD ($)",
)
fig2.show()

In [24]:
city_sold_most = df_city.idxmax()
value_city_sold_max = df_city.max()

#### City sold most product

In [25]:
print("The city with the highest sale is: {}".format(city_sold_most))

The city with the highest sale is: Quantity Ordered     San Francisco
dtype: object


#### Value city most sold

In [26]:
print(value_city_sold_max)

Quantity Ordered    50239.0
dtype: float64


### Question 3


In [27]:
df_time_advertisemens = df[['Quantity Ordered',	'Price Each',"Order Date"]]

In [28]:
df_time_advertisemens['Hour Order'] = df_time_advertisemens['Order Date'].dt.hour

<ipython-input-28-143f6638723d>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
df_time_advertisemens.drop(columns='Order Date',inplace=True)

<ipython-input-29-a2f960cba1d4>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
df_time_advertisemens_group_sum = df_time_advertisemens.groupby("Hour Order").sum()
df_time_advertisemens_group_count = df_time_advertisemens.groupby("Hour Order").count()

In [31]:
fig3 = px.line(x = df_time_advertisemens_group_sum.index,y = df_time_advertisemens_group_sum['Quantity Ordered'])
fig3.add_scatter(x = df_time_advertisemens_group_count.index , y = df_time_advertisemens_group_count['Quantity Ordered'],name = "Count")
fig3.update_layout(
    title="Sum Sales by Hour",
    xaxis_title="Hour",
    yaxis_title="Count",
    legend_title= "Choice count",
)
fig3.show()

In [32]:
df_time_advertisemens_group_sum.sort_values(by='Quantity Ordered',ascending=False)

,Quantity Ordered,Price Each
Hour Order,,
19,14470.0,2398588.31
12,14202.0,2299876.68
11,14005.0,2288855.18
18,13802.0,2207696.93
20,13768.0,2268185.16
13,13685.0,2139743.86
14,12362.0,2072194.77
10,12308.0,1932665.62
21,12244.0,2030763.83


### Questions 4

In [33]:
products_buy = df[["Order ID","Product","Order Date","Purchase Address"]]

In [34]:
order_together = df[["Order ID","Order Date","Purchase Address"]].duplicated(keep = False)

In [35]:
products_buy =  products_buy[order_together]

In [36]:
products_buy

,Order ID,Product,Order Date,Purchase Address
20,248171,USB-C Charging Cable,2019-09-05 15:06:00,"705 Hill St, New York City, NY 10001"
21,248171,USB-C Charging Cable,2019-09-05 15:06:00,"705 Hill St, New York City, NY 10001"
60,248210,iPhone,2019-09-27 10:33:00,"292 Washington St, San Francisco, CA 94016"
61,248210,Lightning Charging Cable,2019-09-27 10:33:00,"292 Washington St, San Francisco, CA 94016"
70,248219,Bose SoundSport Headphones,2019-09-24 12:48:00,"665 Maple St, Los Angeles, CA 90001"
...,...,...,...,...
17520,295526,27in FHD Monitor,2019-11-21 11:26:00,"550 Park St, Dallas, TX 75001"
17528,295534,Google Phone,2019-11-06 19:21:00,"22 Pine St, Boston, MA 02215"
17529,295534,USB-C Charging Cable,2019-11-06 19:21:00,"22 Pine St, Boston, MA 02215"
17594,295599,iPhone,2019-11-25 21:17:00,"258 Ridge St, New York City, NY 10001"


In [37]:
def connect_product(products):
  return ", ".join(products)

In [38]:
products_buy["Grouped Products"] = products_buy.groupby(["Order ID","Order Date","Purchase Address"])["Product"].transform(connect_product)

In [39]:
products_buy = products_buy[["Order ID","Order Date","Purchase Address","Grouped Products"]]

In [40]:
duplicate_index = products_buy.drop_duplicates(subset = ["Order ID","Order Date","Purchase Address"])

In [41]:
product_together = duplicate_index.value_counts("Grouped Products").dropna()

In [42]:
product_together

,count
Grouped Products,
"iPhone, Lightning Charging Cable",882
"Google Phone, USB-C Charging Cable",856
"iPhone, Wired Headphones",361
"Vareebadd Phone, USB-C Charging Cable",312
"Google Phone, Wired Headphones",303
...,...
"LG Dryer, 27in FHD Monitor",1
"LG Dryer, 27in 4K Gaming Monitor",1
"Google Phone, Wired Headphones, USB-C Charging Cable",1


In [43]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in duplicate_index["Grouped Products"].dropna():
    row_list = row.split(", ")
    count.update(Counter(combinations(row_list, 2)))

for key, value in count.most_common(10):
    print(f"{key[0]} and {key[1]} have {value} times soled together")

iPhone and Lightning Charging Cable have 1005 times soled together
Google Phone and USB-C Charging Cable have 987 times soled together
iPhone and Wired Headphones have 447 times soled together
Google Phone and Wired Headphones have 414 times soled together
Vareebadd Phone and USB-C Charging Cable have 361 times soled together
iPhone and Apple Airpods Headphones have 360 times soled together
Google Phone and Bose SoundSport Headphones have 220 times soled together
USB-C Charging Cable and Wired Headphones have 160 times soled together
Vareebadd Phone and Wired Headphones have 143 times soled together
Lightning Charging Cable and Wired Headphones have 92 times soled together


### Question 5


In [47]:
product_sold = df[['Quantity Ordered','Product',"Price Each"]].groupby("Product").sum()

In [49]:
fig5 = px.bar(product_sold)
fig5.update_layout(
    title="Sales by Product",
    xaxis_title="Product",
    yaxis_title="Value",
    legend_title="Choice",
)
fig5.show()